# TESTING (Sobel + Canny)

In [3]:
import os
import cv2
import numpy as np
from skimage.morphology import dilation, square, remove_small_objects
from scipy.ndimage import label
import matplotlib.pyplot as plt

def segment_citrus_leaf(image_path, sobel_kernel_size, dilation_kernel_size, filling_kernel_size):
    # 1. Read and preprocess the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or could not be loaded")
    
    """
    Step 1: Sobel Operator
    """
    # Convert to gray-scale image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Store intermediate results for visualization
    results = {}

    # 1. Edge Detection using Sobel operator
    # The default is 3 (3x3 kernel). 
    # You can increase it to 5 or 7 for smoother edges but less detail, or decrease it for sharper but noisier edges.
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel_size)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel_size)
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    gradient_magnitude = np.uint8(gradient_magnitude * 255 / gradient_magnitude.max())
    results['edges'] = gradient_magnitude

    # plt.imshow(cv2.cvtColor(gradient_magnitude, cv2.COLOR_BGR2RGB))
    
    """
    Step 2: Thresholding
    """
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gradient_magnitude, (5, 5), 0)

    # Use Canny edge detection
    thresh = cv2.Canny(blurred, 0, 255)

    results['threshold'] = thresh

    # plt.imshow(cv2.cvtColor(thresh, cv2.COLOR_BGR2RGB))
    
    """
    Step 3: Dilation
    """
    # 3. Dilation using vertical and horizontal structuring elements
    vertical_se = np.ones((dilation_kernel_size, 1), dtype=np.uint8)
    horizontal_se = np.ones((1, dilation_kernel_size), dtype=np.uint8)
    dilated = cv2.dilate(thresh, vertical_se, iterations=1)
    dilated = cv2.dilate(dilated, horizontal_se, iterations=1)
    results['dilated'] = dilated
    
    """
    Step 4: Filling holes
    """
    # 4. Filling Holes and Removing Small Areas
    # Fill holes using morphological closing
    filled_holes = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, np.ones((filling_kernel_size,filling_kernel_size), dtype=np.uint8))
    results['filled_holes'] = filled_holes
    
    # """
    # Step 5: Find Contours, select largest contour and create the mask
    # """
    # # 1. Find Contours
    contours, _ = cv2.findContours(filled_holes, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 2. Select the largest contour (assuming it's the leaf)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        
    else:
        raise ValueError("No contours found. Check the edge detection step.")

    # 3. Create a Mask from the largest contour
    mask = np.zeros_like(filled_holes)
    cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)
    results['masked'] = mask

    # # plt.imshow(cv2.cvtColor(masked, cv2.COLOR_BGR2RGB))
    
    # """
    # Step 5: Find Contours, select largest contour and create the mask
    # """
    # Make a copy of the edges and define a seed point
    # flood_fill = filled_holes.copy()
    # h, w = flood_fill.shape[:2]
    # mask = np.zeros((h+2, w+2), np.uint8)

    # # Apply the flood fill algorithm
    # cv2.floodFill(flood_fill, mask, (0, 0), 255)

    # # Invert the flood filled image
    # flood_fill_inv = cv2.bitwise_not(flood_fill)

    # # Combine the edges with the flood fill to get the final mask
    # mask = filled_holes | flood_fill_inv
    
    """
    Step 5: Find Contours, select largest contour and create the mask
    """
    # # Find contours in the closed image
    # contours, _ = cv2.findContours(filled_holes, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # Create a blank mask
    # mask = np.zeros_like(filled_holes)

    # # Find the convex hull of the largest contour
    # if contours:
    #     largest_contour = max(contours, key=cv2.contourArea)
    #     hull = cv2.convexHull(largest_contour)

    #     # Draw the convex hull on the mask
    #     cv2.drawContours(mask, [hull], -1, (255), thickness=cv2.FILLED)
    
    """
    Step 6: Segmenting
    """
    segmented = cv2.bitwise_and(image, image, mask=mask)
    results['segmented'] = segmented

    # plt.imshow(cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB))
    
    return segmented, mask

def process_images_in_folders(parent_directory: str, segmented_output_dir: str, final_mask_dir: str,
                              sobel_kernel_size, dilation_kernel_size, filling_kernel_size
                              ):
    # Iterate over each subfolder in the parent directory
    for folder_name in os.listdir(parent_directory):
        folder_path = os.path.join(parent_directory, folder_name)
        if os.path.isdir(folder_path):
            print(f'Processing folder: {folder_name}')
            # Prepare output directories
            segmented_folder = os.path.join(segmented_output_dir, folder_name)
            final_mask_folder = os.path.join(final_mask_dir, folder_name)
            os.makedirs(segmented_folder, exist_ok=True)
            os.makedirs(final_mask_folder, exist_ok=True)

            # Get all image files in the subfolder
            image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
            image_files.sort()  # Sort image files for consistent order

            # Process each image file in the subfolder
            for image_file in image_files:
                image_path = os.path.join(folder_path, image_file)

                # Apply the processing function
                segmented_image, final_mask = segment_citrus_leaf(image_path, sobel_kernel_size, dilation_kernel_size, filling_kernel_size)

                # Save the processed images to the correct output directories
                segmented_output_path = os.path.join(segmented_folder, image_file)
                final_mask_output_path = os.path.join(final_mask_folder, image_file)

                cv2.imwrite(segmented_output_path, segmented_image)
                cv2.imwrite(final_mask_output_path, final_mask)

                print(f'Saved segmented image to: {segmented_output_path}')
                print(f'Saved final mask to: {final_mask_output_path}')

## Indoor

In [5]:
sobel_kernel_size = 5
dilation_kernel_size = 5
filling_kernel_size = 5


# # To test all images
# process_images_in_folders(
#     '/Users/judecao/MUN/Winter2025/DSP/Project/Citrus_indoor/Leaves/',
#     '/Users/judecao/MUN/Winter2025/DSP/Project/Results_indoor/SegmentedOutput/',
#     '/Users/judecao/MUN/Winter2025/DSP/Project/Results_indoor/FinalMask/',
#     sobel_kernel_size, dilation_kernel_size, filling_kernel_size)

# For TESTING Only
process_images_in_folders(
    '/Users/judecao/MUN/Winter2025/DSP/Project/Testing',
    '/Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/SegmentedOutput',
    '/Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/FinalMask',
    sobel_kernel_size, dilation_kernel_size, filling_kernel_size)

# process_images_in_folders(
#     '/Users/judecao/MUN/Winter2025/DSP/Project/citrus_leaves_prepared/validation',
#     '/Users/judecao/MUN/Winter2025/DSP/Project/citrus_leaves_prepared/Results/SegmentedOutput',
#     '/Users/judecao/MUN/Winter2025/DSP/Project/citrus_leaves_prepared/Results/FinalMask',
#     sobel_kernel_size, dilation_kernel_size, filling_kernel_size)


Processing folder: canker
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/SegmentedOutput/canker/1.png
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/FinalMask/canker/1.png
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/SegmentedOutput/canker/10.png
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/FinalMask/canker/10.png
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/SegmentedOutput/canker/131.p.png
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/FinalMask/canker/131.p.png
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/SegmentedOutput/canker/132.p.png
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/FinalMask/canker/132.p.png
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/SegmentedOut

## Outdoor

In [40]:
sobel_kernel_size = 5
dilation_kernel_size = 5
filling_kernel_size = 5


# Example usage
process_images_in_folders(
    '/Users/judecao/MUN/Winter2025/DSP/Project/Citrus_outdoor/Leaves/',
    '/Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/SegmentedOutput/',
    '/Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/FinalMask/',
    sobel_kernel_size, dilation_kernel_size, filling_kernel_size)


Processing folder: canker
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/SegmentedOutput/canker/1.jpg
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/FinalMask/canker/1.jpg
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/SegmentedOutput/canker/10.jpg
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/FinalMask/canker/10.jpg
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/SegmentedOutput/canker/11.jpg
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/FinalMask/canker/11.jpg
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/SegmentedOutput/canker/12.jpg
Saved final mask to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/FinalMask/canker/12.jpg
Saved segmented image to: /Users/judecao/MUN/Winter2025/DSP/Project/Results_outdoor/SegmentedOutput/canker/1

# PERFORMANCE

## preprocess .json

In [10]:
import os
import json
import numpy as np
import cv2
from PIL import Image

def convert_json_to_mask(ground_truth_dir: str):
    # Duyệt qua từng thư mục con (spot, canker,...)
    for folder_name in os.listdir(ground_truth_dir):
        folder_path = os.path.join(ground_truth_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        print(f'\n📂 Processing folder: {folder_name}')
        
        for file in os.listdir(folder_path):
            if file.endswith('.json'):
                json_path = os.path.join(folder_path, file)

                # Lấy tên gốc (loại bỏ cả đuôi .p nếu có)
                base_name = file.replace('.p.json', '').replace('.json', '')
                
                # Tìm ảnh tương ứng (ưu tiên .png, rồi .jpg, .jpeg)
                image_file = None
                for ext in ['.png', '.jpg', '.jpeg']:
                    candidate = base_name + ext
                    candidate_path = os.path.join(folder_path, candidate)
                    if os.path.exists(candidate_path):
                        image_file = candidate
                        break

                if image_file is None:
                    print(f'⚠️ Không tìm thấy ảnh tương ứng cho {file}')
                    continue
                
                image_path = os.path.join(folder_path, image_file)

                try:
                    # Load ảnh
                    image = np.array(Image.open(image_path))
                    height, width = image.shape[:2]
                    mask = np.zeros((height, width), dtype=np.uint8)

                    # Load JSON
                    with open(json_path) as f:
                        data = json.load(f)

                    # Vẽ các shape lên mask
                    for shape in data.get('shapes', []):
                        points = np.array(shape['points'], dtype=np.int32)
                        cv2.fillPoly(mask, [points], color=255)

                    # Lưu mask tại cùng thư mục với ảnh gốc
                    mask_filename = os.path.splitext(image_file)[0] + '.png'
                    mask_path = os.path.join(folder_path, mask_filename)
                    cv2.imwrite(mask_path, mask)

                    print(f'✅ Mask saved to: {mask_path}')
                
                except Exception as e:
                    print(f'❌ Error with {file}: {e}')

convert_json_to_mask('/Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/')


Processing folder: canker
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/48.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/66.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/10.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/34.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/23.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/25.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/31.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/42.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/54.png
✅ Saved mask to: /Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/canker_masks/1

## overal performance

In [9]:
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.metrics import precision_score, jaccard_score, f1_score, recall_score

def evaluate_segmentation_performance(ground_truth_dir: str, predicted_mask_dir: str):
    # Initialize results dictionary
    results = {"canker": [], "greening": [], "healthy": [], "spot": []}
    sample_counts = {}
    
    for folder_name in results.keys():
        gt_folder = os.path.join(ground_truth_dir, folder_name)
        pred_folder = os.path.join(predicted_mask_dir, folder_name)
        
        if os.path.isdir(gt_folder) and os.path.isdir(pred_folder):
            gt_files = sorted([f for f in os.listdir(gt_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
            sample_counts[folder_name] = len(gt_files)
            
            precisions, ious, dices, pixel_accuracies, recalls = [], [], [], [], []
            
            for file_name in gt_files:
                gt_path = os.path.join(gt_folder, file_name)
                pred_path = os.path.join(pred_folder, file_name)
                
                if os.path.exists(pred_path):
                    ground_truth = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
                    predicted_mask = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
                    
                    ground_truth = (ground_truth > 127).astype(np.uint8)
                    predicted_mask = (predicted_mask > 127).astype(np.uint8)
                    
                    ground_truth_flat = ground_truth.flatten()
                    predicted_mask_flat = predicted_mask.flatten()
                    
                    precision = precision_score(ground_truth_flat, predicted_mask_flat, zero_division=1)
                    iou = jaccard_score(ground_truth_flat, predicted_mask_flat, zero_division=1)
                    dice = f1_score(ground_truth_flat, predicted_mask_flat, zero_division=1)
                    recall = recall_score(ground_truth_flat, predicted_mask_flat, zero_division=1)
                    
                    tp = np.sum((predicted_mask == 1) & (ground_truth == 1))
                    tn = np.sum((predicted_mask == 0) & (ground_truth == 0))
                    fp = np.sum((predicted_mask == 1) & (ground_truth == 0))
                    fn = np.sum((predicted_mask == 0) & (ground_truth == 1))
                    pixel_accuracy = (tp + tn) / (tp + tn + fp + fn)
                    
                    precisions.append(precision)
                    ious.append(iou)
                    dices.append(dice)
                    pixel_accuracies.append(pixel_accuracy)
                    recalls.append(recall)
            
            results[folder_name] = [
                np.mean(precisions), np.mean(ious), np.mean(dices), np.mean(pixel_accuracies), np.mean(recalls)
            ]
    
    performance_df = pd.DataFrame(results, index=["Precision", "IoU", "Dice", "Pixel Accuracy", "Recall"])
    sample_counts_df = pd.DataFrame.from_dict(sample_counts, orient='index', columns=['Number of Images'])
    
    
    display("Segmentation Performance", performance_df)
    
    display("Avg Segmentation Performance", performance_df.mean(axis=1))
    
    display("Sample Counts", sample_counts_df)
    
# Example usage
evaluate_segmentation_performance(
    '/Users/judecao/MUN/Winter2025/DSP/Project/GroundTruth_indoor/',
    '/Users/judecao/MUN/Winter2025/DSP/Project/Results_testing/FinalMask/'
)


'Segmentation Performance'

,canker,greening,healthy,spot
Precision,0.801008,0.905342,0.900697,0.851653
IoU,0.655654,0.905335,0.900697,0.816915
Dice,0.728772,0.950286,0.947677,0.881881
Pixel Accuracy,0.833333,0.965279,0.962803,0.938660
Recall,0.732833,0.999991,1.000000,0.941067


'Avg Segmentation Performance'

Precision         0.864675
IoU               0.819650
Dice              0.877154
Pixel Accuracy    0.925019
Recall            0.918473
dtype: float64

'Sample Counts'

,Number of Images
canker,12
greening,4
healthy,8
spot,14
